# Cow behaviour labelling
Using a pre-trained model.

In [1]:
import os

notebook_mode: int = int(
    input(
        """
    Select notebook mode: 
    1. Google Colab  2. Local
    """
    )
)

if notebook_mode == 1:
    # Run on Colab.
    INPUT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/data/"
    SCRIPT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/training_testing/data_labelling/"
    OUTPUT_PATH: str = "/content/drive/MyDrive/Ellinbank/video_observation/output/"
    os.system(command="cp {}custom_model.py .".format(SCRIPT_PATH))
    os.system(command="cp {}inference.py .".format(SCRIPT_PATH))
    os.system(command="cp {}utils.py .".format(SCRIPT_PATH))
    os.system(command="cp {}operation.py .".format(SCRIPT_PATH))
elif notebook_mode == 2:
    INPUT_PATH: str = "../../../data/"
    SCRIPT_PATH: str = "./"
    OUTPUT_PATH: str = "./out/"

In [2]:
import pandas as pd


files: list[str] = os.listdir(path=INPUT_PATH)
files = [f for f in files if f.endswith(".zip")]

sensor_names: list[str] = [name.split("_")[0] for name in files]
print(sensor_names)

['MOS2E03230475']


In [3]:
import random

import tensorflow as tf

window_size: int = 600  # 300: 10 seconds
window_per_epoch: int = 200
epoch: int = 1
batch_size: int = 64
# random.seed(715) # 715 looks good.
random.seed(785)  # 785 makes "other" looks bad, otherwise is good.


model: tf.keras.Model = tf.keras.models.load_model(
    filepath="{}cow_model.keras".format(SCRIPT_PATH)
)

2023-12-11 11:36:09.445722: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 11:36:09.484363: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-11 11:36:09.484415: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-11 11:36:09.484466: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-11 11:36:09.490940: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-12-11 11:36:09.491518: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

In [4]:
# for f in files:
#     data_chunks: pd.DataFrame = pd.read_csv(
#         filepath_or_buffer="{}/{}".format(INPUT_PATH, f),
#     )

# data_chunks.info()

In [5]:
import gc
from pathlib import Path

import numpy as np
import pandas.io.parsers.readers
from operation import clean_data, label_data

for f in files:
    Path("{}{}.txt".format(OUTPUT_PATH, f.split("_")[0])).touch()
    data_chunks: pandas.io.parsers.readers.TextFileReader = pd.read_csv(
        filepath_or_buffer="{}/{}".format(INPUT_PATH, f),
        chunksize=window_size * 10,
    )

    for raw_data in data_chunks:
        full_data: pd.DataFrame = clean_data(full_data=raw_data)
        labels: np.ndarray = label_data(
            raw_data=full_data,
            model=model,
            window_size=window_size,
            batch_size=batch_size,
        )

        with open(
            file="{}{}.txt".format(OUTPUT_PATH, f.split("_")[0]),
            mode="ab",
        ) as file:
            np.savetxt(
                fname=file,
                X=labels,
                fmt="%s",
            )

        gc.collect()